In [ ]:
using Plots
using LinearAlgebra
import Plots: Plot
gr();

In [ ]:
using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
import Plots: Plot
gr();

struct ObsEdge
    t1::Int64
    t2::Int64
    x1::Vector{Float64}
    x2::Vector{Float64}
    z1::Vector{Float64}
    z2::Vector{Float64}
    function ObsEdge(
        t1::Int64,
        t2::Int64,
        z1_::Vector{Float64},
        z2_::Vector{Float64},
        xs::Dict{Int64, Vector{Float64}},
        sensor_noise_rate=[0.14, 0.05, 0.05]
    )
        z1_[1] == z2_[1] || error("Landmark Ids do not match")
        x1 = xs[t1]
        x2 = xs[t2]
        z1 = z1_[2:4]
        z2 = z2_[2:4]
        θ1 = x1[3]
        ϕ1 = z1[2]
        θ2 = x2[3]
        ϕ2 = z2[2]
        s1 = sin(θ1 + ϕ1)
        c1 = cos(θ1 + ϕ1)
        s2 = sin(θ2 + ϕ2)
        c2 = cos(θ2 + ϕ2)
        
        ê = [z2[1] * c2 - z1[1] * c1,
             z2[1] * s2 - z1[1] * s1,
             z2[2] - z2[3] - z1[2] + z1[3]]
        ê = ê + x2 - x1
        while ê[3] >= pi
            ê[3] -= 2*pi
        end
        while ê[3] < -pi
            ê[3] += 2*pi
        end
        
        Q₁ = Diagonal( [(z1[1]*sensor_noise_rate[1])^2, sensor_noise_rate[2]^2, sensor_noise_rate[3]^2] )
        R₁ = [ c1 -z1[1]*s1 0; s1 z1[1]*c1 0; 0 1 -1]
        Q₂ = Diagonal( [(z2[1]*sensor_noise_rate[1])^2, sensor_noise_rate[2]^2, sensor_noise_rate[3]^2] )
        R₂ = [ c2 -z2[1]*s2 0; s2 z2[1]*c2 0; 0 1 -1 ]
        Σ = R₁ * Q₁ * transpose(R₁) + R₂ * Q₂ * transpose(R₂)
        Ω = inv(Σ)
        println(ê)
        println(Σ)
        new(t1, t2, x1, x2, z1, z2)
    end
end

function make_ax(xlim, ylim)
    p = plot(aspect_ratio=:equal, xlim=xlim, ylim=ylim, xlabel="X", ylabel="Y")
    return p
end

function draw_traj!(xs, p::Plot{T}) where {T}
    # xs is dictionary
    poses = [xs[s] for s in 0:length(xs)-1]
    p = scatter!([e[1] for e in poses], [e[2] for e in poses], markershape=:circle, markersize=1, color="black")
    p = plot!([e[1] for e in poses], [e[2] for e in poses], linewidth=0.5, color="black")
end

function draw_observations!(
        xs::Dict{Int64, Vector{Float64}},
        zlist::Dict{Int64, Vector{Vector{Float64}}},
        p::Plot{T}
        ) where {T}
    for s in 0:length(xs)-1
        if !haskey(zlist, s)
            continue
        end
        v = zlist[s]
        N = size(v)[1]
        for i in 1:N
            x, y, θ = xs[s][1], xs[s][2], xs[s][3]
            obsv = v[i]
            d, ϕ = obsv[2], obsv[3]
            mx = x + d * cos(θ + ϕ)
            my = y + d * sin(θ + ϕ)
            p = plot!([x, mx], [y, my], color="pink", alpha=0.5)
        end
    end
end

function draw_edges!(edges::Vector{ObsEdge}, p::Plot{T}) where {T}
    for edge = edges
        p = plot!([edge.x1[1], edge.x2[1]], [edge.x1[2], edge.x2[2]], color="red", alpha=0.5)
    end
end

function draw_all(
    xlim, ylim,
    xs::Dict{Int64, Vector{Float64}},
    zlist::Dict{Int64, Vector{Vector{Float64}}},
    edges::Vector{ObsEdge},
    p::Plot{T}
) where {T}
    p = make_ax(xlim, ylim)
    draw_observations!(xs, zlist, p)
    draw_traj!(xs, p)
    draw_edges!(edges, p)
    return p
end

# hat_xs contains all the keys in 0:length(xs), but z does not (there are some time steps with no observation)
function read_data(fname="log.txt")
    hat_xs = Dict{Int64, Vector{Float64}}()
    zlist = Dict{Int64, Vector{Vector{Float64}}}()
    delta = 0.0
    us = Dict{Int64, Vector{Float64}}()

    fd = open(fname, "r")
    lines = readlines(fd)
    for i in 1:length(lines)
        line = lines[i]
        tokens = split(line, ' ')
        if tokens[1] == "delta"
            delta = parse(Float64, tokens[2])
            continue
        end
        step = parse(Int64, tokens[2])
        if tokens[1] == "x"
            # x, y, θ
            hat_xs[step] = [parse(Float64, tokens[3]), parse(Float64, tokens[4]), parse(Float64, tokens[5])] 
        elseif tokens[1] == "z"
            if !haskey(zlist, step)
                zlist[step] = Vector{Vector{Float64}}(undef, 0)
            end
            # ID of landmark, d, ϕ, m
            z = [parse(Float64, tokens[3]), parse(Float64, tokens[4]), parse(Float64, tokens[5]), parse(Float64, tokens[6])]
            push!(zlist[step], copy(z))
        elseif tokens[1] == "u"
            us[step] = [parse(Float64, tokens[3]), parse(Float64, tokens[4])]
        end
    end
    return hat_xs, zlist
end

function make_edges(
    hat_xs::Dict{Int64, Vector{Float64}},
    zlist::Dict{Int64, Vector{Vector{Float64}}}
)
    landmark_keys_zlist = Dict{Int64, Vector{Vector{Float64}}}()
    
    for (step, zs) in zlist
        for i in 1:lastindex(zs)
            z = zs[i]
            landmark_id = convert(Int64, z[1])
            if !haskey(landmark_keys_zlist, landmark_id)
                landmark_keys_zlist[landmark_id] = Vector{Vector{Float64}}(undef, 0)
            end
            # step, ID of landmark, d, ϕ, m
            push!(landmark_keys_zlist[landmark_id], vcat([step*1.0], z))
        end
    end

    edges = Vector{ObsEdge}(undef, 0)
    
    for (landmark_id, lm_zlist) in landmark_keys_zlist
        for i in 1:lastindex(lm_zlist)
            for j in (i+1):lastindex(lm_zlist)
                z1 = lm_zlist[i]
                z2 = lm_zlist[j]
                t1 = convert(Int64, z1[1])
                t2 = convert(Int64, z2[1])
                push!(edges, ObsEdge(t1, t2, z1[2:5], z2[2:5], hat_xs))
            end
        end
    end

    return edges
end
    
hat_xs, zlist = read_data("log_official.txt");
xlim = [-5.0, 5.0]
ylim = [-5.0, 5.0]
edges = make_edges(hat_xs, zlist)
p = make_ax(xlim, ylim)
p = draw_all(xlim, ylim, hat_xs, zlist, edges, p)
plot(p, legend=nothing)
#savefig(p, "images/ch09_graphslam2.png")